In [2]:
import sys
sys.path.append("..")

In [3]:
from src.preprocess.offens_eval import get_X_and_ys
from src.features.sentiment_score_english import sentiment_score_english
from src.features.keras_padded_w2i import get_padded_w2i_matrix

Using TensorFlow backend.


In [8]:
EN_PATH = "../data/raw/OffensEval2019/start-kit/training-v1/offenseval-training-v1.tsv"
data = get_X_and_ys(EN_PATH)
X = data[0]

sentiment_scores = []
for sentence in X:
    sentiment_scores.append(sentiment_score_english(sentence))
print(sentiment_scores)

X_original = X
y = data[1]
y_mapping = data[4]
max_seq_len = 1000
max_num_words = 20000
X, word_index = get_padded_w2i_matrix(X, max_num_words, max_seq_len)

[0, -1, 1, -1, 0, 0, -1, -1, 0, 1, 1, 1, -1, -1, 1, 0, 1, 1, 1, -1, -1, 0, 1, -1, -1, -1, 0, 0, 1, 1, 1, 1, -1, -1, 0, 1, -1, -1, -1, 0, -1, -1, 1, -1, 1, 1, 1, 1, 0, 1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, 0, 0, 0, -1, -1, -1, -1, 0, 1, 1, -1, 1, 0, -1, 0, -1, 0, 0, -1, 0, -1, 1, 0, 1, 1, 1, -1, 1, 1, 0, 0, -1, 1, 0, 1, 1, 1, -1, -1, 1, -1, 0, 1, -1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1, -1, 1, 0, 1, -1, 0, 1, 1, 1, 0, -1, -1, 0, -1, -1, -1, 0, -1, 0, -1, 0, 1, 0, -1, -1, 1, -1, 0, 1, -1, 0, 1, -1, 1, -1, 0, -1, -1, 1, -1, 1, 0, 1, -1, 1, 0, -1, 1, -1, 0, -1, 1, 1, 1, -1, 0, 1, 1, 1, 0, 0, 0, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 0, 1, 0, -1, -1, 0, -1, -1, -1, -1, -1, 1, 0, -1, 0, -1, 0, -1, 1, 0, 0, 0, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 0, -1, -1, -1, -1, 1, -1, -1, -1, -1, 0, -1, 1, -1, -1, 1, 0, 1, 1, 1, -1, 0, -1, 1, -1, 1, 1, 0, -1, 1, 1, 1, 1, 1, -1, 0, 1, -1, 0, 1, 1, -1, 0, -1, 1, 1, -1, 0, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 0, -1, 1, 0, 0

In [9]:
from src.features.embedding_matrix import get_embedding_matrix, get_embedding_model
emb_dim = 300
from src.CONSTANTS import EN_EMB_FILE_PATH
emb_model = get_embedding_model("../" +EN_EMB_FILE_PATH)
emb_matrix, num_oov = get_embedding_matrix(
    emb_model, 
    emb_dim,
    word_index
)

from src.layers.pretrained_embedding_layer import get_pretrained_embedding_layer

emb_layer = get_pretrained_embedding_layer(
    len(word_index) + 1,
    emb_dim, 
    emb_matrix,
    max_seq_len,
)

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

main_input = Input(shape=(max_seq_len,), dtype='int32', name='main_input')

x = emb_layer(main_input)

lstm_out = LSTM(10)(x)

auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
import keras 

sent_scores = []
for s in sentiment_scores:
    sent_scores.append([s])
    
auxiliary_input = Input(shape=(1,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

x = Dense(8, activation='relu')(x)

In [12]:
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [14]:
from src.preprocess.train_test_val_split import train_test_val_split
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(X, y, test_size=0.2, val_size=0.1, random_state=0)
X_aux_train, y_aux_train, X_aux_test, y_aux_test, X_aux_val, y_aux_val = train_test_val_split(sent_scores, y, test_size=0.2, val_size=0.1, random_state=0)

In [15]:
import numpy as np
hist= model.fit([X_train, np.array(X_aux_train)], [y_train, y_train], epochs=20, batch_size=512)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
9532/9532 [==============================] - 39s 4ms/step - loss: 1.4017 - main_output_loss: 0.7290 - aux_output_loss: 0.6727
Epoch 2/20
9532/9532 [==============================] - 38s 4ms/step - loss: 1.3219 - main_output_loss: 0.6744 - aux_output_loss: 0.6475
Epoch 3/20
9532/9532 [==============================] - 41s 4ms/step - loss: 1.2608 - main_output_loss: 0.6277 - aux_output_loss: 0.6331
Epoch 4/20
9532/9532 [==============================] - 37s 4ms/step - loss: 1.2285 - main_output_loss: 0.6085 - aux_output_loss: 0.6199
Epoch 5/20
9532/9532 [==============================] - 36s 4ms/step - loss: 1.1952 - main_output_loss: 0.5974 - aux_output_loss: 0.5978
Epoch 6/20
9532/9532 [==============================] - 37s 4ms/step - loss: 1.1527 - main_output_loss: 0.5807 - aux_output_loss: 0.5720
Epoch 7/20
9532/9532 [==============================] - 38s 4ms/step - loss: 1.1208 - main_output_loss: 0.5630 - aux_output_loss: 

In [38]:
model.evaluate([X_test, np.array(X_aux_test)], [y_test, y_test])

2648/2648 [==============================] - 19s 7ms/step


[0.9572481914589412, 0.4743762731191975, 0.48287192032056275]

In [7]:
y_pred = model.predict([X_test, np.array(X_aux_test)])
y_pred = np.array(y_pred).flatten()

NameError: name 'model' is not defined

In [6]:
print(y_pred)
exit()
from src.plotting.confusion_matrix import plot_confusion_matrix
y_pred_binary = [0 if value[0] < 0.5 else 1 for value in y_pred]
plot_confusion_matrix(y_test, y_pred_binary, 2, ["NOT", "OFF"])

NameError: name 'y_pred' is not defined

In [58]:
print(y_pred[0].size)
print(len(y_test))

2648
2648
